In [50]:
import re
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd

In [51]:
data_path = Path.cwd() / 'data' / 'processed' / 'win_a.csv'
df = pd.read_csv(data_path)
df

,Location,SSID,Network type,Authentication,Encryption,BSSID,Signal,dBm,Radio type,Channel,Basic rates (Mbps),Other rates (Mbps)
0,Stair2,WLAN-SMU,Infrastructure,WPA2-Enterprise,CCMP,b4:5d:50:fe:41:c1,81%,-59.5,802.11n,6,12 24,36 48 54
1,Stair2,WLAN-SMU,Infrastructure,WPA2-Enterprise,CCMP,38:17:c3:0c:7f:e1,33%,-83.5,802.11n,6,12 24,36 48 54
2,Stair2,WLAN-SMU,Infrastructure,WPA2-Enterprise,CCMP,b4:5d:50:fe:41:91,16%,-92.0,802.11ac,100,12 24,36 48 54
3,Stair2,WLAN-SMU,Infrastructure,WPA2-Enterprise,CCMP,f0:5c:19:ba:bc:d0,22%,-89.0,802.11ac,161,12 24,36 48 54
4,Stair2,WLAN-SMU,Infrastructure,WPA2-Enterprise,CCMP,b4:5d:50:fe:44:a1,29%,-85.5,802.11n,11,12 24,36 48 54
...,...,...,...,...,...,...,...,...,...,...,...,...
2749,SR2-2a,CS168-2096,NaN,NaN,NaN,a0:36:bc:4b:66:40,84%,-58.0,802.11ax,8,1 2 5.5 11,6 9 12 18 24 36 48 54
2750,SR2-2a,i-DRIVE_i12pro_277A95,NaN,NaN,NaN,60:8a:10:ce:6e:42,31%,-84.5,802.11g,2,1 2 5.5 11,6 9 12 18 24 36 48 54
2751,SR2-2a,SMU_5G_178902,NaN,NaN,NaN,90:de:80:27:7a:95,22%,-89.0,802.11n,6,1 2 5.5 11,6 9 12 18 24 36 48 54
2752,SR2-2a,#JyuGae_Guest,NaN,NaN,NaN,a0:36:bc:4b:66:44,67%,-66.5,802.11ax,40,6 24,9 12 18 36 48 54


In [52]:
df['Location'].unique()

array(['Stair2', 'GSR2-4', 'GSR2-3/2', 'GSR2-1', 'PrintingRoom',
       'Stairs1', 'FToilet', 'LW2.1b', 'Lift2', 'Lift1', 'MToilet',
       'Walkway', 'CommonArea', 'Stairs3', 'SR2-4b', 'SR2-4a', 'SR2-3b',
       'GSR2-6', 'SR2-3a', 'SR2-2a'], dtype=object)

In [53]:
df['Location'] = df['Location'].replace('LW2.1a', 'PrintingRoom')
df['Location'].unique()


array(['Stair2', 'GSR2-4', 'GSR2-3/2', 'GSR2-1', 'PrintingRoom',
       'Stairs1', 'FToilet', 'LW2.1b', 'Lift2', 'Lift1', 'MToilet',
       'Walkway', 'CommonArea', 'Stairs3', 'SR2-4b', 'SR2-4a', 'SR2-3b',
       'GSR2-6', 'SR2-3a', 'SR2-2a'], dtype=object)

In [54]:
location_counts = df['Location'].value_counts().reset_index()
location_counts.columns = ['Location', 'Row Count']

location_counts

,Location,Row Count
0,SR2-4b,214
1,SR2-4a,201
2,SR2-3b,190
3,GSR2-6,186
4,GSR2-4,176
5,Stair2,163
6,CommonArea,152
7,Stairs3,142
8,GSR2-3/2,135
9,SR2-3a,134


In [55]:
df.columns

Index(['Location', 'SSID', 'Network type', 'Authentication', 'Encryption',
       'BSSID', 'Signal', 'dBm', 'Radio type', 'Channel', 'Basic rates (Mbps)',
       'Other rates (Mbps)'],
      dtype='object')

In [60]:
filtered_df = df[['Location', 'SSID', 'BSSID', 'dBm', 'Radio type', 'Channel']]
filtered_df

,Location,SSID,BSSID,dBm,Radio type,Channel
0,Stair2,WLAN-SMU,b4:5d:50:fe:41:c1,-59.5,802.11n,6
1,Stair2,WLAN-SMU,38:17:c3:0c:7f:e1,-83.5,802.11n,6
2,Stair2,WLAN-SMU,b4:5d:50:fe:41:91,-92.0,802.11ac,100
3,Stair2,WLAN-SMU,f0:5c:19:ba:bc:d0,-89.0,802.11ac,161
4,Stair2,WLAN-SMU,b4:5d:50:fe:44:a1,-85.5,802.11n,11
...,...,...,...,...,...,...
2749,SR2-2a,CS168-2096,a0:36:bc:4b:66:40,-58.0,802.11ax,8
2750,SR2-2a,i-DRIVE_i12pro_277A95,60:8a:10:ce:6e:42,-84.5,802.11g,2
2751,SR2-2a,SMU_5G_178902,90:de:80:27:7a:95,-89.0,802.11n,6
2752,SR2-2a,#JyuGae_Guest,a0:36:bc:4b:66:44,-66.5,802.11ax,40


In [61]:
filtered_df = filtered_df.rename(columns={
    'Location': 'location',
    'SSID': 'ssid',
    'BSSID': 'bssid',
    'dBm': 'dbm',
    'Radio type': 'radio_type',
    'Channel': 'channel',
    'Authentication': 'authentication',
    'Encryption': 'encryption',
    'Network type': 'network type',
})

filtered_df

,location,ssid,bssid,dbm,radio_type,channel
0,Stair2,WLAN-SMU,b4:5d:50:fe:41:c1,-59.5,802.11n,6
1,Stair2,WLAN-SMU,38:17:c3:0c:7f:e1,-83.5,802.11n,6
2,Stair2,WLAN-SMU,b4:5d:50:fe:41:91,-92.0,802.11ac,100
3,Stair2,WLAN-SMU,f0:5c:19:ba:bc:d0,-89.0,802.11ac,161
4,Stair2,WLAN-SMU,b4:5d:50:fe:44:a1,-85.5,802.11n,11
...,...,...,...,...,...,...
2749,SR2-2a,CS168-2096,a0:36:bc:4b:66:40,-58.0,802.11ax,8
2750,SR2-2a,i-DRIVE_i12pro_277A95,60:8a:10:ce:6e:42,-84.5,802.11g,2
2751,SR2-2a,SMU_5G_178902,90:de:80:27:7a:95,-89.0,802.11n,6
2752,SR2-2a,#JyuGae_Guest,a0:36:bc:4b:66:44,-66.5,802.11ax,40


In [62]:
output_path = Path.cwd() / 'data' / 'processed' / 'win_e_processed.csv'
filtered_df.to_csv(output_path, index=False)
filtered_df

,location,ssid,bssid,dbm,radio_type,channel
0,Stair2,WLAN-SMU,b4:5d:50:fe:41:c1,-59.5,802.11n,6
1,Stair2,WLAN-SMU,38:17:c3:0c:7f:e1,-83.5,802.11n,6
2,Stair2,WLAN-SMU,b4:5d:50:fe:41:91,-92.0,802.11ac,100
3,Stair2,WLAN-SMU,f0:5c:19:ba:bc:d0,-89.0,802.11ac,161
4,Stair2,WLAN-SMU,b4:5d:50:fe:44:a1,-85.5,802.11n,11
...,...,...,...,...,...,...
2749,SR2-2a,CS168-2096,a0:36:bc:4b:66:40,-58.0,802.11ax,8
2750,SR2-2a,i-DRIVE_i12pro_277A95,60:8a:10:ce:6e:42,-84.5,802.11g,2
2751,SR2-2a,SMU_5G_178902,90:de:80:27:7a:95,-89.0,802.11n,6
2752,SR2-2a,#JyuGae_Guest,a0:36:bc:4b:66:44,-66.5,802.11ax,40


In [63]:
filtered_df.describe()

,dbm,channel
count,2754.000000,2754.000000
mean,-83.099129,43.067538
std,8.763189,50.296532
min,-96.000000,1.000000
25%,-89.000000,6.000000
50%,-85.500000,11.000000
75%,-80.000000,64.000000
max,-56.500000,161.000000
